In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

if IS_COLAB:
    %pip install -q -U tensorflow-addons
    %pip install -q -U transformers

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "nlp"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


# 1. Char-RNN을 사용해 셰익스피어 문체 생성하기
- https://github.com/karpathy/char-rnn 셰익스피어 작품을 모두 다운로드 가능


## 1.1 훈련 데이터셋 만들기

In [2]:
# 자료 다운

shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)

with open(filepath) as f:
    shakespeare_text = f.read()

1130496/1115394 [==============================] - 0s 0us/step


In [3]:
# 모든 글자를 정수로 인코딩해야 함
# 사용자 정의 전처리 층을 만드는 것이 한 방법, 여기에서는 더 간단하게 케라스의 Tokenizer클래스를 사용
# 이 클래스는 기본적으로 텍스트를 소문자로 바꿈, (원치 않는 경우 lower=False로 지정)
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True) 
# char_level=True로 지정하여 단어 수준 인코딩 대신 글자수준 인코딩을 만듬

tokenizer.fit_on_texts(shakespeare_text)

- 이제 문자을 (또는 문장의 리스트를 ) 그랒 ID로 인코딩하거나 반대로 디코딩할 수 있음. 
- 이를 통해 텍스트에 있는 고유 글자 개수와 전체 글자 개수를 알 수 있음

In [4]:
tokenizer.texts_to_sequences(["first"])

[[20, 6, 9, 8, 3]]

In [6]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [9]:
max_id = len(tokenizer.word_index) # 고유 글자 개수
dataset_size = tokenizer.document_count # 전체 글자 개수

In [10]:
# 전체 텍스트를 인코딩하여 각 글자를 1D로 나타내 봄
# 1 ~ 39까지 대신 0 ~ 38까지 1D를 얻기 위해 1을 빼줌

[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

## 1. 2 순차 데이터셋을 나누는 방법
- 훈련세트, 검증세트, 테스트 세트가 중복되지 않도록 만드는 것이 중요!

In [11]:
# 90%를 훈련 세트로 사용
# 한 번에 한 그랒식 반환하는 tf.data.Dataset 객체를 만듬

train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

## 1.3 순차 데이터를 윈도 여러 개로 자르기
- 훈련세트는 백만 개 이상의 글자로 이루어진 시퀀스 하나임. 여기에 신경망을 직접 훈련시킬 수 없음
- 데이터셋의 window() 메서드를 사용해 이 긴 시퀀스를 작은 많은 텍스트 윈도로 변환합니다.
- 이 데이터셋의 각 샘플은 전체 텍스트엣 매우 짧은 부분 문자열임
- RNN은 이 부분 문자열 길이만큼만 역전파를 위해 펼쳐짐. 이를 TBPTT(Truncated BackPropagation Through Time)

In [12]:
# window() 메서드를 호출하여 짧은 텍스트 윈도를 갖는 데이터셋을 만듬

n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

# tip : n_steps는 튜닝 가능, 짧은 입력 시퀀스에서 RNN을 훈련하는 것은 쉽지만 당연히 이 RNN은 n_steps보다 긴 패턴을 학습할 수 없음 
# 따라서 너무 짧게 만들어서는 안된다.
# shift = 1로 지정하면 가장 큰 훈련세트를 만들 수 있음

In [13]:
dataset = dataset.flat_map(lambda window: window.batch(window_length)) # 윈도마다 batch(window_lenght)를 호출

In [15]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows:(windows[:, :-1], windows[:, 1:]))

- 일반저으로 범주형 입력 특성은 원-핫 벡터나 임베딩으로 인코딩 되어야 함. 여기에서는 고유한 글자수가 적기 때문에(39개) 원-핫 벡터를
- 글자를 인코딩합니다.

In [16]:
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

--- 
- 데이터셋 준비 완료!

## 1.4 Char-RNN 모델 만들고 훈련하기
- 이전 글자 100개를 기반으로 다음 글자를 예측하기 우해 유닛 128개를 가진 GRU층 2개와 입력(dropout)과 은닉 상태(recurrent_dropout)에 20% 드롭아웃을 사용
- 출력층은 TimeDistributed 클래스르 적용한 Dense 층입니다.
- 텍스트에 있는고유한 글자 수가 39개이므로 이 층은 39개의 유닛(max_id)을 가져야 함

In [19]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=2)

Epoch 1/2
31368/31368 [==============================] - 4462s 142ms/step - loss: 1.6194
Epoch 2/2
31368/31368 [==============================] - 4230s 135ms/step - loss: 1.5395


## 1.5 Char-RNN 모델 사용하기
- 위 모델에 새로운 텍스트를 주입하려면 앞에서와 같은 전처리를 해야 한다.

In [20]:
# 텍스트 전처리 함수

def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [25]:
# 모델을 사용해 어떤 텍스트이 다음 글자를 예측

X_new = preprocess(["How are yo"])
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 첫 번째 문장, 마지막 글자


# 정확하게 맞힌것을 확인할 수 있음

'u'

## 1.6 가짜 셰익스피어 텍스트를 생성하기


In [26]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

# 그 다음 next_char()함수를 반복 호출하여 다음 글자를 얻고 텍스트에 추가하는 작음 함수를 만듬

In [27]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [28]:
# 온도를 다르게 하며 테스트 해보기

print(complete_text("t", temperature=0.2))

the more to her and servant,
i will be good and sig


In [29]:
print(complete_text("w", temperature=1))

wtruns, my lord, and was the worship no clows
to gu


In [30]:
print(complete_text("w", temperature=2))

wore. i bise! i know he.
fayh, nih,
standsirousewsc


- 이 셰익스피어 모델은 1에 가까운 온도에서 가장 잘 작동됨. 조금 더 좋은 텍스트를 생성하려면 GRU 층과 층의 뉴런 수를 늘리고 더 오래 훈련하거나 규제(예를 들어 GRU 층을 recurrent_dropout=0.3으로 지정할 수 있음)를 추가. 
- 윈도를 크게할 수 있지만 훈련이 더 어려워짐

## 1.7 상태가 있는 RNN
- RNN이 한 훈련 배치를 처리한 후에 마지막 상태를 다음 훈련 배치의 초기상태로 사용하면 어떨까??
- 이렇게 하면 역전파는 짧은 시퀀스에서 일어나지만 모델이 장기간 패턴을 학습할 수 있음 이를 **상태가 있는 RNN**이라고 함

In [31]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [32]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [33]:
# 상태가 있는 RNN 모델

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2,
                     dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

- 에포크 끝마다 테스트를 다시 시작하기 전에 상태를 재설정해야 함. 콜백함수를 사용하여 처리

In [34]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [35]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50,
                    callbacks=[ResetStatesCallback()])

Epoch 1/50
313/313 [==============================] - 28s 81ms/step - loss: 2.6242
Epoch 2/50
313/313 [==============================] - 28s 88ms/step - loss: 2.2457
Epoch 3/50
313/313 [==============================] - 32s 101ms/step - loss: 2.1168
Epoch 4/50
313/313 [==============================] - 37s 119ms/step - loss: 2.0396
Epoch 5/50
313/313 [==============================] - 37s 119ms/step - loss: 1.9880
Epoch 6/50
313/313 [==============================] - 35s 113ms/step - loss: 1.9510
Epoch 7/50
313/313 [==============================] - 37s 117ms/step - loss: 1.9238
Epoch 8/50
313/313 [==============================] - 37s 118ms/step - loss: 1.8997
Epoch 9/50
313/313 [==============================] - 36s 114ms/step - loss: 1.8798
Epoch 10/50
313/313 [==============================] - 36s 114ms/step - loss: 1.8637
Epoch 11/50
313/313 [==============================] - 36s 116ms/step - loss: 1.8525
Epoch 12/50
313/313 [==============================] - 38s 120ms/step - loss

---
- 이 모델을 훈련한 후에 훈련할 때 사용한 거솨 동일한 크기의 배치로만 예측을 만들 수 있음. 이런 제약을 없애려면 동일한 구조의 상태가 없는 모델을 만들고 상태가 있는 모델의 가중치를 복사한다.

In [36]:
# tip

stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

stateless_model.build(tf.TensorShape([None, None, max_id]))
stateless_model.set_weights(model.get_weights())
model = stateless_model

tf.random.set_seed(42)

print(complete_text("t"))

tidy.

menenius:
mercuie, it than shall live,
the c


# 2. 감성 분석
- 영화 리뷰가 부정적인지(0), 긍정적인지(1)

In [37]:
# 전처리되어 있는 데이터셋

(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()
X_train[0][:10]

17473536/17464789 [==============================] - 1s 0us/step


[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

- 0, 1, 2는 각각 패딩 토큰 SOS 토큰 알수 없는 단어를 의미함

In [38]:
# 리뷰 내용을 보고 싶다면 다음과 같이 디코딩

word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

1654784/1641221 [==============================] - 0s 0us/step


'<sos> this film was just brilliant casting location scenery story'

---

- 텐서플로 데이터셋을 사용해 원본 IMDb 리뷰를 텍스트(바이트 스트링)으로 적재

In [40]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling imdb_reviews-train.tfrecord...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling imdb_reviews-test.tfrecord...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling imdb_reviews-unsupervised.tfrecord...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to C:\Users\cheol\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [43]:
test_size = info.splits["test"].num_examples

In [44]:
datasets.keys()

dict_keys([Split('train'), Split('test'), Split('unsupervised')])

In [45]:
train_size, test_size

(25000, 25000)

In [48]:
# 전처리 함수를 만든다.

def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ") # 태그를 공백으로 바꿈
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ") # 대문자 소문자
    X_batch = tf.strings.split(X_batch)
    
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

- 리뷰 텍스트를 잘라내어 각 리뷰에서 처음 300 글자만 남김. 이렇게 하면 훈련 속도를 높일 수 있음

In [50]:
for X_batch, y_batch in datasets["train"].batch(2).take(1):
    for review, label in zip(X_batch.numpy(), y_batch.numpy()):
        print("Review:", review.decode("utf-8")[:200], "...")
        print("Label:", label, "= Positive" if label else "= Negative")
        print()

Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0 = Negative

Review: I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0 = Negative



In [51]:
preprocess(X_batch, y_batch)

(<tf.Tensor: shape=(2, 53), dtype=string, numpy=
 array([[b'This', b'was', b'an', b'absolutely', b'terrible', b'movie',
         b"Don't", b'be', b'lured', b'in', b'by', b'Christopher',
         b'Walken', b'or', b'Michael', b'Ironside', b'Both', b'are',
         b'great', b'actors', b'but', b'this', b'must', b'simply', b'be',
         b'their', b'worst', b'role', b'in', b'history', b'Even',
         b'their', b'great', b'acting', b'could', b'not', b'redeem',
         b'this', b"movie's", b'ridiculous', b'storyline', b'This',
         b'movie', b'is', b'an', b'early', b'nineties', b'US',
         b'propaganda', b'pi', b'<pad>', b'<pad>', b'<pad>'],
        [b'I', b'have', b'been', b'known', b'to', b'fall', b'asleep',
         b'during', b'films', b'but', b'this', b'is', b'usually', b'due',
         b'to', b'a', b'combination', b'of', b'things', b'including',
         b'really', b'tired', b'being', b'warm', b'and', b'comfortable',
         b'on', b'the', b'sette', b'and', b'having', b'j

In [52]:
#  Counter로 단어의 등장 횟수를 셈

from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [53]:
# 가장 많이 등장한 단어 확인

vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [54]:
# 어휘 사전 중에서 가장 많이 등장한 단어 10000개만 남기고 삭제

vocab_size = 10000
truncated_vocabulary = [word for word, count in vocabulary.most_common()[:vocab_size]]

In [56]:
# 각 단어를 1D(즉, 어휘 사전의 인덱스)로 바꾸는 전처리 단계를 추가

words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [59]:
# 테이블에서 단어 몇 개에 대한 1D를 확인해 봄

table.lookup(tf.constant(b"This movie was faaaantastic".split()))

# faaaantastic은 없기 때문에 10000보다 크거나 같은 1D를 가진 00V 버킷 중 하나에 매핑됨

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([   22,    12,    11, 10771], dtype=int64)>

- 리뷰를 배치로 묶고  preprocess()함수를 사용해 단어으 짧은 시퀀스로 바꿈
- 그다음 앞서 만든 테이블을 사용하는 encode-words() 함수로 단어를 인코딩함
- 마지막으로 다음 배치를 프리페치 함

In [60]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

# 훈련 세트 준비
train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [62]:
# 모델을 만들어 훈련

embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, input_shape=[None]), # 임베딩으로 변환하는 embedding 층
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 65s 76ms/step - loss: 0.5389 - accuracy: 0.7154
Epoch 2/5
782/782 [==============================] - 80s 102ms/step - loss: 0.3348 - accuracy: 0.8600
Epoch 3/5
782/782 [==============================] - 79s 101ms/step - loss: 0.1849 - accuracy: 0.9337
Epoch 4/5
782/782 [==============================] - 79s 101ms/step - loss: 0.1298 - accuracy: 0.9552
Epoch 5/5
782/782 [==============================] - 81s 103ms/step - loss: 0.1178 - accuracy: 0.9560


## 2.1 마스킹
- 패딩 토큰을 무시하도록 모데에게 알려주어 실제 의미가 있는 데이터에 집중할 수 있게 만드는 것
- Embedding 층을 만들 때 mask_zero= True 매개변수 추가

In [ ]:
K = keras.backend
embed_size = 128
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)
model = keras.models.Model(inputs=[inputs], outputs=[outputs])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(train_set, epochs=5)

## 사전훈련된 임베딩 재사용하기

In [ ]:
tf.random.set_seed(42)

TFHUB_CACHE_DIR = os.path.join(os.curdir, "my_tfhub_cache")
os.environ["TFHUB_CACHE_DIR"] = TFHUB_CACHE_DIR

import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]), # 모듈 다운로드, 이 모듈의 이름은 "문장 인코더"
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam",
              metrics=["accuracy"])


for dirpath, dirnames, filenames in os.walk(TFHUB_CACHE_DIR):
    for filename in filenames:
        print(os.path.join(dirpath, filename))

In [ ]:
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].batch(batch_size).prefetch(1)
history = model.fit(train_set, epochs=5)

# 3. 신경망 기계 번역을 위한 인코더-디코더 네트워크

In [66]:
vocab_size = 100
embed_size = 10

In [67]:
import tensorflow_addons as tfa

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler,
                                                 output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, initial_state=encoder_state,
    sequence_length=sequence_lengths)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.models.Model(
    inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
    outputs=[Y_proba])

In [68]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [69]:
X = np.random.randint(100, size=10*1000).reshape(1000, 10)
Y = np.random.randint(100, size=15*1000).reshape(1000, 15)
X_decoder = np.c_[np.zeros((1000, 1)), Y[:, :-1]]
seq_lengths = np.full([1000], 15)

history = model.fit([X, X_decoder, seq_lengths], Y, epochs=2)

Epoch 1/2
32/32 [==============================] - 6s 111ms/step - loss: 4.6052
Epoch 2/2
32/32 [==============================] - 3s 106ms/step - loss: 4.6031


## 3.1 양방향 RNN
- keras에서 양방향 순환 층ㅇ르 구현하려면 keras.layers.Bidirectional으로 순환 층을 감쌉니다.

In [70]:


model = keras.models.Sequential([
    keras.layers.GRU(10, return_sequences=True, input_shape=[None, 10]),
    keras.layers.Bidirectional(keras.layers.GRU(10, return_sequences=True)) 
])

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_14 (GRU)                (None, None, 10)          660       
                                                                 
 bidirectional (Bidirectiona  (None, None, 20)         1320      
 l)                                                              
                                                                 
Total params: 1,980
Trainable params: 1,980
Non-trainable params: 0
_________________________________________________________________


# 4. 어텐션 메커니즘